In [ ]:
include("init.jl")
using TensorOperations
using TensorDecompositions
using Statistics
using LinearAlgebra
import Plots
import StatsPlots
import LightGraphs

In [ ]:
δ(args...) = reduce(==,args) |> Int
⊗(a,b) = kron(a,b)

function CONCORDIA(X,factors,R)
    us = []
    ss = []
    vs = []
    for f in factors
        (u,s,v) = svd(f)
        push!(us,u)
        push!(ss,s)
        push!(vs,v)
    end

    y = mapreduce(u -> u |> transpose, ⊗, us)*X[:]
    z = mapreduce(s -> s |> Diagonal |> inv, ⊗, ss)*y
    inds = [R for _ in 1:length(factors)]
    G = reshape(reduce(⊗,vs)*z,inds...)


    mapreduce(k -> (G[k] - δ(Tuple(k)...))^2,+,pairs(IndexCartesian(),G)|>keys)
    #[G[i,j,k] - δ(i,j,k) for i in 1:R, j in 1:R, k in 1:R].^2 |> sum
    # how the dickens do we reduce this?
end

In [ ]:
DIMS = [100,100,10,10]
u,v,w,x = [rand(dim) for dim in DIMS]
z = rand(DIMS...)
X = zeroes(DIMS...)

# This is using Einstein's summation convention - the whole tensor a is generated by iterating through all possible pairs 
@tensor X[i,j,k,l] = u[i] * v[j] * w[k] * x[l]#+ z[i,j,k]
X |> size

In [ ]:
#F = candecomp(a, 1, (randn(10,1),randn(20,1),randn(30,1));
#    # NB: need to have 2D arrays for the guess; 1D isn't good enough
#    compute_error=true,
#    method=:ALS,
#    maxiter=100_000,
#);
for r in 1:3
    F = nncp(X, r;
        compute_error=true,
        maxiter=100,
    );
    @show CONCORDIA(X,F.factors,r)
end

In [ ]:
# In retrospect, we don't actually need this at all - normal kron is fine? :(

struct KronProd{T} <: AbstractMatrix{T}
    A::AbstractMatrix{T}
    B::AbstractMatrix{T}
end

import Base.size
function size(K::KronProd{T})::Tuple{Int,Int} where T
    return size(K.A).*size(K.B)
end

import Base.getindex
function getindex(K::KronProd{T},i::Int,j::Int) where T
    (m::Int,n::Int) = size(K.A)
    (p::Int,q::Int) = size(K.B)
    a_ind = (floor((i-1)/p + 1) |> Int,floor((j-1)/q + 1) |> Int)
    b_ind = (i - floor((i-1)/p)*p |> Int, j - floor((j-1)/q)*q |> Int)
    return @inbounds K.A[a_ind...] * K.B[b_ind...]
end

#⊗(a,b) = KronProd(a,b) # called \otimes

In [ ]:
pairs(IndexCartesian(),[2 2; 1 3])

In [ ]:
methods(enumerate)